In [1]:
#Mounting Google Drive from Google Colab
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#Changing the current working directory to the Google Drive
#%cd /content/drive/My Drive/MLDL2024_project1-Enrico

In [3]:
#!pip install -U fvcore

In [4]:
#Importing the necessary libraries
import os
import torch
import numpy as np
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from datasets.cityscapes import CityscapesCustom
from datasets.gta5 import GTA5Custom
from models.deeplabv2.deeplabv2 import get_deeplab_v2
from models.bisenet.build_bisenet import BiSeNet
from train import train_model
from utils import test_latency_FPS, test_FLOPs_params, plot_miou_over_epochs

In [5]:
#Set device agnostic code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Set the manual seeds
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

#Set training parameters
cityscapes_height, cityscapes_width = (512, 1024)
gta5_height, gta5_width = (720, 1280)
train_batch_size = 4
n_epochs = 10

class_names = ["road", "sidewalk", "building", "wall", "fence", "pole", "traffic_light", "traffic_sign",
                "vegetation", "terrain", "sky", "person", "rider", "car", "truck", "bus", "train",
                "motorcycle", "bicycle"
]

In [6]:
#Create Dataloaders for Cityscapes
cityscapes_dir = os.path.dirname(os.getcwd()) + '/Cityscapes/Cityspaces/'

cityscapes_train_dataset = CityscapesCustom(cityscapes_dir, 'train', cityscapes_height, cityscapes_width)
cityscapes_test_dataset = CityscapesCustom(cityscapes_dir, 'val', cityscapes_height, cityscapes_width)

cityscapes_train_dataloader = DataLoader(cityscapes_train_dataset, train_batch_size, shuffle=True)
cityscapes_test_dataloader = DataLoader(cityscapes_test_dataset, train_batch_size, shuffle=False)

print(f'Train: {len(cityscapes_train_dataset)} images, divided into {len(cityscapes_train_dataloader)} batches of size {cityscapes_train_dataloader.batch_size}')
print(f'Test: {len(cityscapes_test_dataset)} images, divided into {len(cityscapes_test_dataloader)} batches of size {cityscapes_test_dataloader.batch_size}')

Train: 1572 images, divided into 393 batches of size 4
Test: 500 images, divided into 125 batches of size 4


In [7]:
#Create Dataloader for GTA5
gta5_dir = os.path.dirname(os.getcwd()) + '/GTA5/GTA5/'

gta5_dataset = GTA5Custom(gta5_dir, gta5_height, gta5_width)

gta5_dataloader = DataLoader(gta5_dataset, train_batch_size, shuffle=True)

print(f'GTA5: {len(gta5_dataset)} images, divided into {len(gta5_dataloader)} batches of size {gta5_dataloader.batch_size}')

GTA5: 2500 images, divided into 625 batches of size 4


In [8]:
image, label = gta5_dataset[0]
#Count how many pixel are in each class for the first label
unique, counts = np.unique(label, return_counts=True)
print(dict(zip(unique, counts)))
#print the type of the image and the label
print(type(image), type(label))

{0: 305340, 2: 203543, 3: 30937, 4: 1278, 5: 3923, 8: 9330, 9: 1979, 10: 42724, 11: 96073, 14: 15349, 255: 211124}
<class 'torch.Tensor'> <class 'torch.Tensor'>


In [9]:
unique_labels = []
dataloader = cityscapes_test_dataset

for i, (image, label) in enumerate(dataloader):
    if i % 40 == 0:
        print(f'Batch {i}/{len(dataloader)}')

    if i == 120:
        break
    
    for lab in label.unique():
        if lab not in unique_labels:
            unique_labels.append(lab)

unique_labels

Batch 0/500
Batch 40/500
Batch 80/500
Batch 120/500


[tensor(0),
 tensor(1),
 tensor(2),
 tensor(4),
 tensor(5),
 tensor(7),
 tensor(8),
 tensor(10),
 tensor(11),
 tensor(13),
 tensor(255),
 tensor(12),
 tensor(18),
 tensor(6),
 tensor(9),
 tensor(14),
 tensor(17),
 tensor(16),
 tensor(15),
 tensor(3)]

In [10]:
len(unique_labels)

20

In [ ]:
#Testing Classic semantic segmentation network

#Set up the model with the pretrained weights
DeepLabV2_model = get_deeplab_v2().to(device)

#Set up the loss function and the optimizer
DeepLabV2_criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
DeepLabV2_optimizer = optim.Adam(DeepLabV2_model.parameters(), lr=1e-3)

#Test latency and FPS
#print(test_latency_FPS(DeepLabV2_model, device, cityscapes_height, cityscapes_width))

#Test FLOPs and number of parameters
#print(test_FLOPs_params(DeepLabV2_model, device, cityscapes_height, cityscapes_width))

#Train the model
all_train_miou, all_test_miou, best_epoch = train_model(DeepLabV2_model, DeepLabV2_criterion, DeepLabV2_optimizer,
            cityscapes_train_dataloader, cityscapes_test_dataloader, class_names, device, n_epochs,
            lr_schedule=False, model_name='DeepLabV2')

plot_miou_over_epochs(all_train_miou, all_test_miou, best_epoch)

In [17]:
#Testing Real-Time semantic segmentation network

#Set up the model with the pretrained weights
BiSeNet_model = BiSeNet(num_classes=19, context_path='resnet18').to(device)

#Set up the loss function and the optimizer for BiSeNet
BiSeNet_criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
BiSeNet_optimizer = torch.optim.SGD(BiSeNet_model.parameters(), lr=2.5e-2, momentum=0.9, weight_decay=1e-4)

#Test latency and FPS
#print(test_latency_FPS(BiSeNet_model, device, cityscapes_height, cityscapes_width))

#Test FLOPs and number of parameters
#print(test_FLOPs_params(BiSeNet_model, device, cityscapes_height, cityscapes_width))

#Train the model
all_train_miou, all_test_miou, best_epoch = train_model(BiSeNet_model, BiSeNet_criterion, BiSeNet_optimizer,
            cityscapes_train_dataloader, cityscapes_test_dataloader, class_names, device, n_epochs,
            lr_schedule=True, model_name='BiSeNet')

plot_miou_over_epochs(all_train_miou, all_test_miou, best_epoch)

KeyboardInterrupt: 

In [ ]:
#Testing Domain Shift problem in semantic segmentation

#Set up the model with the pretrained weights
BiSeNet_model = BiSeNet(num_classes=19, context_path='resnet18').to(device)

#Set up the loss function and the optimizer for BiSeNet
BiSeNet_criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
BiSeNet_optimizer = torch.optim.SGD(BiSeNet_model.parameters(), lr=2.5e-2, momentum=0.9, weight_decay=1e-4)

#Train the model
all_train_miou, all_test_miou, best_epoch = train_model(BiSeNet_model, BiSeNet_criterion, BiSeNet_optimizer,
            gta5_dataloader, cityscapes_test_dataloader, class_names, device, n_epochs,
            lr_schedule=True, model_name='BiSeNet')

plot_miou_over_epochs(all_train_miou, all_test_miou, best_epoch)

IndexError: Target 23 is out of bounds.